# Generate log file

In [1]:
%matplotlib inline
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
import re

import pandas as pd

## List all logs files

In [2]:
import glob
files = glob.glob('../results/logs/**/events.out.tfevents.*', recursive=True)
#files = glob.glob('../results/logs_2019_03_09 _RBM_RBMCF_100_250_500_1000/**/events.out.tfevents.*', recursive=True)

len(files), files[0:3]

(17,
 ['../results/logs/kfold=2/kfold-intern=0/batch_size=10/class=RBMCF/visible_size=702/hidden_size=10000/regularization=NoRegularization-0.0/learning_rate=Adam-0.05-0.9-0.999-1e-08/sampling_method=CD-1/momentum=0/1557963054.2556417/events.out.tfevents.1557963062.paulo-notebook',
  '../results/logs/kfold=2/kfold-intern=0/batch_size=10/class=RBMCF/visible_size=702/hidden_size=1000/regularization=NoRegularization-0.0/learning_rate=Adam-0.05-0.9-0.999-1e-08/sampling_method=CD-1/momentum=0/1557084657.908597/events.out.tfevents.1557084664.paulo-notebook',
  '../results/logs/kfold=2/kfold-intern=0/batch_size=10/class=RBMCF/visible_size=702/hidden_size=1000/regularization=NoRegularization-0.0/learning_rate=Adam-0.05-0.9-0.999-1e-08/sampling_method=CD-1/momentum=0/1557117393.224676/events.out.tfevents.1557117398.paulo-notebook'])

## Read a file

In [3]:
file = files[0]

def extract_information(file):
    ea = EventAccumulator(file)
    ea.Reload()
    
    scalars = ea.Tags()['scalars']
    first_scalar = ea.Scalars(scalars[0])
    
    dataframes = pd.DataFrame(first_scalar)[['wall_time', 'step']]
    
    for df_name in ea.Tags()['scalars']:
        df = pd.DataFrame(ea.Scalars(df_name))
        series = df['value'].rename(df_name)
        dataframes = pd.concat([dataframes, series], axis=1)
    
    del ea

    return dataframes

dfs = extract_information(file)
print(dfs.shape[0], 'evaluates')
print(dfs.shape[1], 'data columns')

dfs.head()

51 evaluates
38 data columns


,wall_time,step,measure/reconstruction/hamming,details/measure/top-1/evaluate-0/train,details/measure/top-1/evaluate-0/validation,details/measure/top-1/evaluate-1/train,details/measure/top-1/evaluate-1/validation,details/measure/top-1/evaluate-2/train,details/measure/top-1/evaluate-2/validation,details/measure/top-1/evaluate-3/train,...,details/measure/top-5/evaluate-5/validation,measure/evaluate/top-5/train,measure/evaluate/top-5/validation,measure/evaluate/Free/mean_free_energy_train,measure/evaluate/Free/mean_free_energy_validation,measure/evaluate/Free/ratio_mean_free_energy,measure/evaluate/Free/mean_free_energy_gap,measure/evaluate/Free/mean_free_energy_noisy,measure/evaluate/reconstruction/RMSE_train,measure/evaluate/reconstruction/RMSE_validation
0,1.557963e+09,1,2.580622,0.000723,0.002890,0.000723,0.014451,0.029646,0.031792,0.084599,...,0.676301,0.400578,0.405588,-421.821686,-420.956329,0.997949,26.548401,-394.407928,0.101541,0.101088
1,1.557963e+09,2,2.783080,0.300072,0.358382,0.010846,0.028902,0.138829,0.095376,0.084599,...,0.667630,0.422029,0.434489,-253.673080,-252.470535,0.995259,30.782043,-221.688492,0.088012,0.087482
2,1.557963e+09,3,2.900217,0.187274,0.138728,0.096168,0.092486,0.126537,0.086705,0.084599,...,0.676301,0.443601,0.452794,-181.968124,-180.988037,0.994614,23.078156,-157.909882,0.087841,0.087692
3,1.557963e+09,4,2.929862,0.300072,0.358382,0.043384,0.060694,0.126537,0.086705,0.157628,...,0.710983,0.456857,0.460983,-133.510681,-132.860413,0.995129,19.221756,-113.638657,0.087696,0.087523
4,1.557963e+09,5,2.908894,0.300072,0.358382,0.096168,0.092486,0.085322,0.098266,0.157628,...,0.710983,0.451796,0.455684,-107.226242,-106.793770,0.995967,17.789062,-89.004707,0.087602,0.087132


In [4]:
def extract_columns(file):
    regex = r'(.+?)=(.+?)\/'
    test_str = file.replace('../results/logs/', '')
    matches = re.finditer(regex, test_str, re.MULTILINE)

    columns = {match.groups()[0]: match.groups()[1] for match in matches}
    columns['path'] = test_str
    return columns

def populate_columns(file, dataframe):
    columns = extract_columns(file)
    
    for k, v in columns.items():
        dataframe[k] = v

    return dataframe

In [5]:
def generate_logs_files(files):
    data = []

    for index, file in enumerate(files):
        print(f'{index+1} of {len(files)}')
        information = extract_information(file)
        information = populate_columns(file, information)
        
        data.append(information)
        
    return pd.concat(data)

data = generate_logs_files(files)

1 of 17
2 of 17
3 of 17
4 of 17
5 of 17
6 of 17
7 of 17
8 of 17
9 of 17
10 of 17
11 of 17
12 of 17
13 of 17
14 of 17
15 of 17
16 of 17
17 of 17


In [6]:
data.index.name = 'evaluate'
data.head(5)

,wall_time,step,measure/reconstruction/hamming,details/measure/top-1/evaluate-0/train,details/measure/top-1/evaluate-0/validation,details/measure/top-1/evaluate-1/train,details/measure/top-1/evaluate-1/validation,details/measure/top-1/evaluate-2/train,details/measure/top-1/evaluate-2/validation,details/measure/top-1/evaluate-3/train,...,kfold-intern,batch_size,class,visible_size,hidden_size,regularization,learning_rate,sampling_method,momentum,path
evaluate,,,,,,,,,,,,,,,,,,,,,
0,1.557963e+09,1,2.580622,0.000723,0.002890,0.000723,0.014451,0.029646,0.031792,0.084599,...,0,10,RBMCF,702,10000,NoRegularization-0.0,Adam-0.05-0.9-0.999-1e-08,CD-1,0,kfold=2/kfold-intern=0/batch_size=10/class=RBM...
1,1.557963e+09,2,2.783080,0.300072,0.358382,0.010846,0.028902,0.138829,0.095376,0.084599,...,0,10,RBMCF,702,10000,NoRegularization-0.0,Adam-0.05-0.9-0.999-1e-08,CD-1,0,kfold=2/kfold-intern=0/batch_size=10/class=RBM...
2,1.557963e+09,3,2.900217,0.187274,0.138728,0.096168,0.092486,0.126537,0.086705,0.084599,...,0,10,RBMCF,702,10000,NoRegularization-0.0,Adam-0.05-0.9-0.999-1e-08,CD-1,0,kfold=2/kfold-intern=0/batch_size=10/class=RBM...
3,1.557963e+09,4,2.929862,0.300072,0.358382,0.043384,0.060694,0.126537,0.086705,0.157628,...,0,10,RBMCF,702,10000,NoRegularization-0.0,Adam-0.05-0.9-0.999-1e-08,CD-1,0,kfold=2/kfold-intern=0/batch_size=10/class=RBM...
4,1.557963e+09,5,2.908894,0.300072,0.358382,0.096168,0.092486,0.085322,0.098266,0.157628,...,0,10,RBMCF,702,10000,NoRegularization-0.0,Adam-0.05-0.9-0.999-1e-08,CD-1,0,kfold=2/kfold-intern=0/batch_size=10/class=RBM...


In [7]:
data.to_csv('extracted_log-validation.csv')